### Load tensorflow

In [0]:
import tensorflow as tf

### Enable Eager Execution if you are using tensorflow 1.x

In [0]:
#Enable Eager Execution
#tf.enable_eager_execution()

### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('./prices2.csv',encoding='utf-8')

### Check all columns in the dataset

In [73]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data=data.drop(['date','symbol'],axis=1)

In [100]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [76]:
data.shape

(999, 5)

### Convert Float64 to Float32

In [101]:
data.dtypes

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Divide the data into train and test sets

In [0]:
X=data.drop('close', axis=1)
y=data["close"]

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 1)

In [0]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

### Normalize Train and Test Data 

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

In [88]:
X_train

array([[8.51502318e-06, 8.32890146e-06, 8.54493464e-06, 1.00000000e+00],
       [9.02353895e-06, 8.79468189e-06, 9.10527433e-06, 1.00000000e+00],
       [1.27343546e-05, 1.26587793e-05, 1.30163062e-05, 1.00000000e+00],
       ...,
       [1.04472536e-04, 1.03533064e-04, 1.04582476e-04, 1.00000000e+00],
       [1.26646424e-04, 1.23368736e-04, 1.27374806e-04, 9.99999940e-01],
       [1.16344170e-04, 1.14913542e-04, 1.18731979e-04, 9.99999940e-01]],
      dtype=float32)

## Building the graph in tensorflow

2.Define Weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [0]:
import numpy as np
y_train=np.array(y_train)

In [0]:
y_train.shape

In [111]:

#Train for 100 Steps
for i in range(100):  
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 7228.8174
Current Loss on iteration 1 6920.2485
Current Loss on iteration 2 6635.868
Current Loss on iteration 3 6373.7905
Current Loss on iteration 4 6132.2554
Current Loss on iteration 5 5909.6553
Current Loss on iteration 6 5704.5107
Current Loss on iteration 7 5515.4463
Current Loss on iteration 8 5341.2056
Current Loss on iteration 9 5180.6255
Current Loss on iteration 10 5032.6353
Current Loss on iteration 11 4896.247
Current Loss on iteration 12 4770.5527
Current Loss on iteration 13 4654.711
Current Loss on iteration 14 4547.952
Current Loss on iteration 15 4449.56
Current Loss on iteration 16 4358.8896
Current Loss on iteration 17 4275.321
Current Loss on iteration 18 4198.3057
Current Loss on iteration 19 4127.326
Current Loss on iteration 20 4061.9194
Current Loss on iteration 21 4001.6333
Current Loss on iteration 22 3946.076
Current Loss on iteration 23 3894.8738
Current Loss on iteration 24 3847.6868
Current Loss on iteration 25 3804.1973
Curre

### Get the shapes and values of W and b

In [113]:
w.shape

TensorShape([Dimension(4), Dimension(1)])

In [114]:
b.shape

TensorShape([Dimension(1)])

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [116]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
699/699 [==============================] - 0s 511us/sample - loss: 5440.7169
Epoch 2/100
699/699 [==============================] - 0s 34us/sample - loss: 3664.8299
Epoch 3/100
699/699 [==============================] - 0s 43us/sample - loss: 3361.0834
Epoch 4/100
699/699 [==============================] - 0s 32us/sample - loss: 3309.2247
Epoch 5/100
699/699 [==============================] - 0s 30us/sample - loss: 3299.4108
Epoch 6/100
699/699 [==============================] - 0s 32us/sample - loss: 3297.1965
Epoch 7/100
699/699 [==============================] - 0s 32us/sample - loss: 3295.7333
Epoch 8/100
699/699 [==============================] - 0s 32us/sample - loss: 3296.5574
Epoch 9/100
699/699 [==============================] - 0s 35us/sample - loss: 3295.0857
Epoch 10/100
699/699 [==============================] - 0s 30us/sample - loss: 3295.1854
Epoch 11/100
699/699 [==============================] - 0s 30us/sample - loss: 3294.5331
Epoch 12/100
699/699 [=======

### Classification using Keras 

In [117]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


### Load the given Iris data using pandas (Iris.csv)

In [0]:

iris =  pd.read_csv('Iris.csv',encoding='utf-8')



In [154]:
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
iris=iris.drop('Id', axis=1)

### Splitting the data into feature set and target set

In [0]:
X=iris.drop('Species', axis=1)
y=iris['Species']

In [144]:
y.unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
from sklearn import preprocessing 
new_le = preprocessing.LabelEncoder()
y = new_le.fit_transform(y)
#

In [151]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### Divide the dataset into Training and test (70:30)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 1)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

In [160]:
X_train.shape

(105, 4)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape((16,),input_shape=(4,4,)))

#Normalize the data
#model.add(tf.keras.layers.BatchNormalization())
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, input_shape=(4,)))
#Add 2st hidden layer
model.add(tf.keras.layers.Dense(8, input_shape=(4,)))

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Fitting the model and predicting 

In [178]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=60,
          batch_size=9)

Train on 105 samples, validate on 45 samples
Epoch 1/60
105/105 [==============================] - 0s 278us/sample - loss: 0.2802 - acc: 0.9619 - val_loss: 0.3711 - val_acc: 0.7778
Epoch 2/60
105/105 [==============================] - 0s 220us/sample - loss: 0.2715 - acc: 0.9333 - val_loss: 0.3316 - val_acc: 0.8667
Epoch 3/60
105/105 [==============================] - 0s 201us/sample - loss: 0.2569 - acc: 0.9714 - val_loss: 0.3539 - val_acc: 0.7778
Epoch 4/60
105/105 [==============================] - 0s 194us/sample - loss: 0.2581 - acc: 0.9333 - val_loss: 0.3100 - val_acc: 0.9111
Epoch 5/60
105/105 [==============================] - 0s 253us/sample - loss: 0.2461 - acc: 0.9524 - val_loss: 0.3085 - val_acc: 0.8889
Epoch 6/60
105/105 [==============================] - 0s 215us/sample - loss: 0.2388 - acc: 0.9333 - val_loss: 0.2900 - val_acc: 0.9333
Epoch 7/60
105/105 [==============================] - 0s 214us/sample - loss: 0.2503 - acc: 0.9238 - val_loss: 0.2644 - val_acc: 0.9778
Epo

### Report Accuracy of the predicted values

In [0]:
pred = model.predict(X_test[1:20])

In [169]:
pred

array([[0.37042573, 0.40065044, 0.22892383],
       [0.24995834, 0.39650226, 0.35353938],
       [0.7742466 , 0.21264888, 0.01310452],
       [0.10263067, 0.31914976, 0.57821953],
       [0.21938878, 0.40067595, 0.3799353 ],
       [0.16302341, 0.36791566, 0.4690609 ],
       [0.6887999 , 0.28072137, 0.03047874],
       [0.6708743 , 0.28860673, 0.04051897],
       [0.10345713, 0.31265786, 0.583885  ],
       [0.20710367, 0.37474608, 0.41815028],
       [0.7310393 , 0.2483173 , 0.02064342],
       [0.07929769, 0.26867604, 0.6520263 ],
       [0.24552867, 0.4048546 , 0.34961677],
       [0.19751579, 0.37680864, 0.42567548],
       [0.6989368 , 0.26931038, 0.03175285],
       [0.23734517, 0.38144067, 0.3812142 ],
       [0.19149338, 0.3797887 , 0.42871794],
       [0.72247684, 0.2522405 , 0.02528261],
       [0.7331748 , 0.24278864, 0.02403661]], dtype=float32)

In [180]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 27        
Total params: 165
Trainable params: 165
Non-trainable params: 0
_________________________________________________________________


In [183]:
model.get_weights()

[array([[-0.31788787,  0.43388194,  0.2879032 , -0.12187522, -0.90959394,
         -0.45839992,  0.26836815,  0.11660837,  0.5676788 , -0.6161493 ],
        [-0.15442742,  0.30753347, -0.42175385, -0.38223737, -0.32476994,
          0.07140361, -0.5257383 , -0.18139294,  0.253847  , -0.15285009],
        [-0.13562317,  0.20282999, -0.5108787 ,  0.9104387 ,  0.821052  ,
          0.29916242, -0.22498618,  0.507541  ,  0.11608257,  0.28612366],
        [-0.5739226 , -0.5035594 , -0.6797633 ,  0.8924591 , -0.2238059 ,
          0.18313545,  0.21523964,  0.6917666 , -0.4652084 , -0.04097715]],
       dtype=float32),
 array([ 0.03454738,  0.0392665 ,  0.06934653, -0.12198906, -0.15337941,
         0.04497722,  0.0291349 , -0.12994203,  0.06417608, -0.02317879],
       dtype=float32),
 array([[ 0.30848673,  0.01191197, -0.31364635, -0.12531932,  0.00123977,
         -0.54083985, -0.05801965,  0.2542783 ],
        [ 0.07822305, -0.37351492,  0.2978684 ,  0.07796657,  0.21430863,
         -0.1

In [0]:
model.save('iris.h5')

In [0]:
from google.colab import files

files.download('iris.h5')

In [188]:
# evaluate the model
model.evaluate(X_test, y_test, verbose=0)

[0.11222029162777794, 0.9777778]

#Accuracy is 97 Percent